<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/150_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voting в ансамблевых моделях


## 🟢 Зеленый уровень — Базовое понимание

**Ключевая идея:**  
Ансамбль — это когда мы берём несколько моделей и объединяем их прогнозы, чтобы получить более устойчивый и точный результат.  
**Voting** — самый простой способ объединить результаты, похожий на «голосование» в компании друзей:  
- Если модели выдают классы — побеждает тот класс, за который «проголосовало» большинство (Majority Voting).  
- Если модели предсказывают числа — берём среднее значение (Averaging).  

**Пример с жизнью:**  
Представь, что трое друзей угадывают цену ноутбука. Один сказал 1000$, другой 1200$, третий 1100$. Среднее — 1100$ — и это наш итоговый прогноз.  

**Два основных вида Voting:**
1. **Hard Voting** — берём класс, набравший больше всего голосов.
2. **Soft Voting** — берём среднее по вероятностям (и потом выбираем класс с наибольшей усреднённой вероятностью).



**🟢 Упражнения:**
1. У тебя есть 3 модели классификации, которые предсказывают кошка/собака:  
   - M1: кошка, M2: собака, M3: собака.  
   Кто победит в hard voting?
2. Если модели выдают вероятности принадлежности к «собаке»: 0.3, 0.7, 0.8 — кто победит в soft voting?
3. Реализуй функцию `hard_vote(predictions)` на Python, которая принимает список предсказаний и возвращает результат голосования.


In [1]:

from collections import Counter

def hard_vote(predictions):
    counts = Counter(predictions)
    return counts.most_common(1)[0][0]

# Пример использования:
preds = ["cat", "dog", "dog"]
print(hard_vote(preds))


dog



## 🟡 Жёлтый уровень — Чуть глубже

**Проблема одной модели:**  
- Может быть переобучение.
- Может быть нестабильна к шуму в данных.

**Почему Voting помогает:**  
- Ошибки отдельных моделей могут «гасить» друг друга.
- Если модели разнообразны (разные алгоритмы, разные признаки, разные обучающие выборки), то объединение почти всегда работает лучше, чем любая модель по отдельности.

**Soft vs Hard Voting — нюанс:**  
Soft Voting обычно работает лучше, **если** модели умеют выдавать вероятности, потому что учитывает уверенность.  
Hard Voting игнорирует уверенность, но работает, даже если модели выдают только «метки».

**Когда Voting не помогает:**
- Если все модели одинаковые и делают одинаковые ошибки.
- Если модели слабо лучше случайного угадывания (тогда ошибки не гасятся).


In [4]:

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Пример: Soft Voting в sklearn
clf1 = LogisticRegression()
clf2 = DecisionTreeClassifier()
clf3 = SVC(probability=True)

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3)],
    voting='soft'
)

# eclf.fit(X_train, y_train)
# print(eclf.score(X_test, y_test))


NameError: name 'X_train' is not defined


**🟡 Упражнения:**
1. Объясни, почему Voting особенно полезен, если у моделей высокая дисперсия ошибок.
2. Придумай способ сделать модели более разнообразными перед голосованием.
3. Измени код выше так, чтобы использовать hard voting, и проверь, как изменится точность.



## 🔴 Красный уровень — Для любопытных и амбициозных

**Математическая идея (без страшных формул):**  
Пусть вероятность, что одна модель ошибается, — `p`.  
Если ошибки независимы, то вероятность, что ансамбль из `n` моделей (при нечётном `n`) ошибётся, равна:

$$
P_{ensemble} = \sum_{k = \frac{n+1}{2}}^n \binom{n}{k} p^k (1-p)^{n-k}
$$

Это биномиальное распределение — оно показывает, что даже если `p < 0.5`, ансамбль может сильно снизить ошибку.

**Интересный момент:**  
- Voting — это частный случай **Bagging**, если обучаем одинаковые алгоритмы на разных подвыборках и просто голосуем.
- Можно сделать **взвешенное голосование** (Weighted Voting), когда у каждой модели свой вес, пропорциональный её точности.
- В реальности ошибки моделей часто коррелированы, поэтому реальное улучшение меньше, чем в идеальной теории.

**Продвинутая идея:**  
Voting — это **не обязательно про классификацию**.  
В регрессии мы можем брать среднее, медиану или даже более сложные агрегаторы (например, усечённое среднее, которое игнорирует крайние значения).


In [5]:

import math

def ensemble_error_prob(p, n):
    total = 0
    for k in range((n+1)//2, n+1):
        total += math.comb(n, k) * (p**k) * ((1-p)**(n-k))
    return total

# Пример: p=0.3, n=3
print(ensemble_error_prob(0.3, 3))


0.216



**🔴 Упражнения:**
1. Если `p = 0.3`, `n = 3`, посчитай вероятность ошибки ансамбля при независимых ошибках.
2. Реализуй взвешенное голосование для трёх моделей с весами 0.5, 0.3, 0.2.
3. Придумай пример, когда soft voting даст неправильный результат, хотя hard voting даст правильный.
4. Поиграйтесь с синтетическими данными и оцените, как корреляция ошибок между моделями влияет на выигрыш от Voting.
